In [ ]:
# === Setup ===
import numpy as np
import matplotlib.pyplot as plt
import torch
from pathlib import Path
import sys
import time

sys.path.insert(0, str(Path.cwd().parent.parent.parent))

from modules._import_helper import safe_import_from

set_seed = safe_import_from('00_repo_standards.src.mlphys_core', 'set_seed')
(HeatEquationConfig, HeatEquationPINN,
 solve_heat_equation_finite_difference) = safe_import_from(
    '07_physics_informed_ml.src.pde_pinn',
    'HeatEquationConfig', 'HeatEquationPINN',
    'solve_heat_equation_finite_difference'
)

# Matplotlib config
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 11
plt.rcParams['axes.grid'] = True
plt.rcParams['grid.alpha'] = 0.3

reports_dir = Path.cwd().parent / 'reports'
reports_dir.mkdir(exist_ok=True)

SEED = 42
set_seed(SEED)
torch.manual_seed(SEED)

print(f"PyTorch version: {torch.__version__}")
print(f"Reports will be saved to: {reports_dir}")
print("✓ Setup complete")

---
## 1. The Physics Problem

### 1D Heat Equation

$$\frac{\partial u}{\partial t} = \alpha \frac{\partial^2 u}{\partial x^2}$$

where:
- $u(x, t)$ is temperature
- $\alpha$ is thermal diffusivity (m²/s)

**Domain**: $x \in [0, L]$, $t \in [0, T]$

**Boundary conditions** (Dirichlet):
$$u(0, t) = u_L, \quad u(L, t) = u_R$$

**Initial condition**:
$$u(x, 0) = u_0(x)$$

### Physical Intuition

- **Large α**: Fast diffusion (heat spreads quickly)
- **Small α**: Slow diffusion (heat persists longer)
- **Steady state**: $u \to$ linear profile when $u_L \neq u_R$, or constant when $u_L = u_R$

In [ ]:
# === Finite Difference Baseline ===
# Setup problem
alpha = 0.01  # Thermal diffusivity
L = 1.0       # Domain length
T_max = 1.0   # Simulation time
bc_left, bc_right = 0.0, 0.0  # Dirichlet BCs

# Grid
Nx, Nt = 50, 100
x_fd = np.linspace(0, L, Nx)
t_fd = np.linspace(0, T_max, Nt)

# Initial condition: Gaussian bump
def u0_gaussian(x, center=0.5, width=0.1):
    return np.exp(-((x - center)**2) / (2 * width**2))

# Solve with finite difference
print("Solving with finite difference (Crank-Nicolson)...")
start_time = time.time()
u_fd = solve_heat_equation_finite_difference(
    alpha=alpha, x=x_fd, t=t_fd,
    u0_fn=u0_gaussian,
    bc_left=bc_left, bc_right=bc_right
)
fd_time = time.time() - start_time
print(f"✓ FD solved in {fd_time:.4f}s")

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Time snapshots
ax = axes[0]
time_indices = [0, Nt//4, Nt//2, 3*Nt//4, -1]
colors = plt.cm.viridis(np.linspace(0, 1, len(time_indices)))
for i, t_idx in enumerate(time_indices):
    ax.plot(x_fd, u_fd[t_idx], color=colors[i], lw=2, label=f't={t_fd[t_idx]:.2f}')
ax.set_xlabel('Position x')
ax.set_ylabel('Temperature u')
ax.set_title('Solution Evolution (FD)')
ax.legend()

# Heatmap
ax = axes[1]
X_fd, T_fd = np.meshgrid(x_fd, t_fd)
im = ax.pcolormesh(X_fd, T_fd, u_fd, shading='auto', cmap='hot')
plt.colorbar(im, ax=ax, label='u(x,t)')
ax.set_xlabel('Position x')
ax.set_ylabel('Time t')
ax.set_title('Solution Field (FD)')

plt.tight_layout()
plt.savefig(reports_dir / '02_heat_fd_baseline.png', dpi=150, bbox_inches='tight')
plt.show()

---
## 2. PINN Architecture for PDE

### Network
$$\hat{u}(x, t) = \text{MLP}([x, t]; \theta)$$

Input: $(x, t)$ ∈ ℝ²  
Output: $\hat{u}$ ∈ ℝ

### Physics Loss

$$\mathcal{L}_{\text{PDE}} = \frac{1}{N_c} \sum_{i=1}^{N_c} \left| \frac{\partial \hat{u}}{\partial t} - \alpha \frac{\partial^2 \hat{u}}{\partial x^2} \right|^2$$

Computed via autograd:
```python
u = model(xt)  # xt requires_grad=True
u_t = grad(u, xt)[..., 1]  # ∂u/∂t
u_x = grad(u, xt)[..., 0]  # ∂u/∂x
u_xx = grad(u_x, xt)[..., 0]  # ∂²u/∂x²
```

### Boundary & Initial Condition Losses

$$\mathcal{L}_{\text{BC}} = \frac{1}{N_b} \sum_{j=1}^{N_b} \left[ (\hat{u}(0, t_j) - u_L)^2 + (\hat{u}(L, t_j) - u_R)^2 \right]$$

$$\mathcal{L}_{\text{IC}} = \frac{1}{N_i} \sum_{k=1}^{N_i} (\hat{u}(x_k, 0) - u_0(x_k))^2$$

### Total Loss

$$\mathcal{L} = \lambda_{\text{PDE}} \mathcal{L}_{\text{PDE}} + \lambda_{\text{BC}} \mathcal{L}_{\text{BC}} + \lambda_{\text{IC}} \mathcal{L}_{\text{IC}}$$

In [ ]:
# === Train PINN ===
set_seed(SEED)

config = HeatEquationConfig(
    alpha=alpha,
    x_min=0.0, x_max=L,
    t_max=T_max,
    bc_left=bc_left, bc_right=bc_right,
    initial_condition="gaussian",
    n_collocation_x=50,
    n_collocation_t=50,
    n_boundary=50,
    n_initial=50,
    hidden_dims=[64, 64, 64, 64],
    epochs=8000,
    lr=1e-3,
    lambda_physics=1.0,
    lambda_bc=10.0,
    lambda_ic=10.0,
)

pinn = HeatEquationPINN(config)

print(f"Network: 2 → {config.hidden_dims} → 1")
print(f"Collocation: {config.n_collocation_x}x{config.n_collocation_t} = {config.n_collocation_x * config.n_collocation_t} points")
print(f"Loss weights: λ_PDE={config.lambda_physics}, λ_BC={config.lambda_bc}, λ_IC={config.lambda_ic}")
print("\nTraining...")

start_time = time.time()
history = pinn.train(verbose=2000)
pinn_time = time.time() - start_time

print(f"\n✓ Training completed in {pinn_time:.2f}s")
print(f"Final loss: {history['loss'][-1]:.6f}")

In [ ]:
# === Compare PINN vs FD ===
# Evaluate PINN on same grid
X_grid, T_grid = np.meshgrid(x_fd, t_fd)
x_flat = X_grid.flatten()
t_flat = T_grid.flatten()

u_pinn_flat = pinn.predict(x_flat, t_flat)
u_pinn = u_pinn_flat.reshape(X_grid.shape)

# Compute errors
error = np.abs(u_pinn - u_fd)
rmse = np.sqrt(np.mean(error**2))
max_error = np.max(error)
rel_error = rmse / np.std(u_fd) * 100

print("="*60)
print("PINN vs Finite Difference")
print("="*60)
print(f"RMSE:           {rmse:.6f}")
print(f"Max Error:      {max_error:.6f}")
print(f"Relative Error: {rel_error:.2f}%")
print("-"*60)
print(f"PINN time:      {pinn_time:.2f}s")
print(f"FD time:        {fd_time:.4f}s")
print(f"Speedup (FD):   {pinn_time/fd_time:.1f}x slower")
print("="*60)

In [ ]:
# === Visualization ===
fig, axes = plt.subplots(2, 3, figsize=(15, 9))

# Training convergence
ax = axes[0, 0]
ax.semilogy(history['loss'], 'k-', lw=2, label='Total')
ax.semilogy(history['loss_physics'], 'b--', lw=1.5, alpha=0.7, label='PDE')
ax.semilogy(history['loss_bc'], 'g-.', lw=1.5, alpha=0.7, label='BC')
ax.semilogy(history['loss_ic'], 'r:', lw=1.5, alpha=0.7, label='IC')
ax.set_xlabel('Epoch')
ax.set_ylabel('Loss')
ax.set_title('Training Convergence')
ax.legend()

# FD solution
ax = axes[0, 1]
im = ax.pcolormesh(X_grid, T_grid, u_fd, shading='auto', cmap='hot')
plt.colorbar(im, ax=ax, label='u')
ax.set_xlabel('x'); ax.set_ylabel('t')
ax.set_title('Finite Difference Solution')

# PINN solution
ax = axes[0, 2]
im = ax.pcolormesh(X_grid, T_grid, u_pinn, shading='auto', cmap='hot')
plt.colorbar(im, ax=ax, label='u')
ax.set_xlabel('x'); ax.set_ylabel('t')
ax.set_title('PINN Solution')

# Error heatmap
ax = axes[1, 0]
im = ax.pcolormesh(X_grid, T_grid, error, shading='auto', cmap='Reds')
plt.colorbar(im, ax=ax, label='|Error|')
ax.set_xlabel('x'); ax.set_ylabel('t')
ax.set_title(f'Absolute Error (Max: {max_error:.4f})')

# Time snapshots comparison
ax = axes[1, 1]
for t_idx in [0, Nt//2, -1]:
    ax.plot(x_fd, u_fd[t_idx], 'k-', lw=2, alpha=0.7)
    ax.plot(x_fd, u_pinn[t_idx], 'r--', lw=2)
ax.plot([], [], 'k-', label='FD')
ax.plot([], [], 'r--', label='PINN')
ax.set_xlabel('x'); ax.set_ylabel('u')
ax.set_title('Solution Profiles (t=0, 0.5, 1.0)')
ax.legend()

# Error over time
ax = axes[1, 2]
error_vs_time = np.sqrt(np.mean(error**2, axis=1))
ax.plot(t_fd, error_vs_time, 'r-', lw=2)
ax.set_xlabel('Time t')
ax.set_ylabel('RMSE at time t')
ax.set_title('Error Accumulation Over Time')

plt.tight_layout()
plt.savefig(reports_dir / '02_pinn_vs_fd_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

---
## 3. Experiment: Sampling Strategy

**Question**: Does random vs uniform sampling of collocation points matter?

In [ ]:
# === Experiment: Sampling strategies ===
# Note: Current implementation uses random sampling
# We test different densities

n_samples_list = [(20, 20), (30, 30), (50, 50), (70, 70)]
results_sampling = []

print("Experiment: Collocation Density")
print("="*60)

for n_x, n_t in n_samples_list:
    set_seed(SEED)
    
    config_test = HeatEquationConfig(
        alpha=alpha, x_min=0.0, x_max=L, t_max=T_max,
        bc_left=bc_left, bc_right=bc_right,
        initial_condition="gaussian",
        n_collocation_x=n_x, n_collocation_t=n_t,
        n_boundary=50, n_initial=50,
        hidden_dims=[64, 64, 64],
        epochs=5000,
        lr=1e-3,
    )
    
    pinn_test = HeatEquationPINN(config_test)
    _ = pinn_test.train(verbose=0)
    
    u_pred = pinn_test.predict(x_flat, t_flat).reshape(X_grid.shape)
    rmse_test = np.sqrt(np.mean((u_pred - u_fd)**2))
    
    results_sampling.append({
        'n_total': n_x * n_t,
        'n_x': n_x, 'n_t': n_t,
        'rmse': rmse_test,
    })
    
    print(f"N_col={n_x}x{n_t}={n_x*n_t:4d} | RMSE={rmse_test:.6f}")

# Plot
fig, ax = plt.subplots(figsize=(8, 5))
n_totals = [r['n_total'] for r in results_sampling]
rmses = [r['rmse'] for r in results_sampling]
ax.loglog(n_totals, rmses, 'ko-', lw=2, ms=10)
ax.set_xlabel('Total Collocation Points')
ax.set_ylabel('RMSE')
ax.set_title('Accuracy vs Collocation Density')
plt.tight_layout()
plt.savefig(reports_dir / '02_exp_sampling_density.png', dpi=150, bbox_inches='tight')
plt.show()

---
## 4. Experiment: Diffusivity Coefficient Scaling

**Question**: How does α affect PINN training and accuracy?

In [ ]:
# === Experiment: Different diffusivity values ===
alpha_values = [0.001, 0.01, 0.05, 0.1]
results_alpha = []

print("Experiment: Effect of Diffusivity α")
print("="*60)

fig, axes = plt.subplots(2, 2, figsize=(12, 10))
axes = axes.flatten()

for idx, alpha_test in enumerate(alpha_values):
    set_seed(SEED)
    
    # FD baseline for this alpha
    u_fd_test = solve_heat_equation_finite_difference(
        alpha=alpha_test, x=x_fd, t=t_fd,
        u0_fn=u0_gaussian, bc_left=0.0, bc_right=0.0
    )
    
    # Train PINN
    config_test = HeatEquationConfig(
        alpha=alpha_test,
        x_min=0.0, x_max=L, t_max=T_max,
        bc_left=0.0, bc_right=0.0,
        initial_condition="gaussian",
        n_collocation_x=50, n_collocation_t=50,
        hidden_dims=[64, 64, 64],
        epochs=5000,
        lr=1e-3,
    )
    
    pinn_test = HeatEquationPINN(config_test)
    history_test = pinn_test.train(verbose=0)
    
    u_pred = pinn_test.predict(x_flat, t_flat).reshape(X_grid.shape)
    rmse_test = np.sqrt(np.mean((u_pred - u_fd_test)**2))
    
    # Characteristic diffusion time
    t_diff = L**2 / alpha_test
    
    results_alpha.append({
        'alpha': alpha_test,
        't_diff': t_diff,
        'rmse': rmse_test,
        'final_loss': history_test['loss'][-1],
    })
    
    # Plot
    ax = axes[idx]
    im = ax.pcolormesh(X_grid, T_grid, np.abs(u_pred - u_fd_test), shading='auto', cmap='Reds')
    plt.colorbar(im, ax=ax, label='|Error|')
    ax.set_xlabel('x'); ax.set_ylabel('t')
    ax.set_title(f'α={alpha_test} (τ_diff={t_diff:.1f})\nRMSE={rmse_test:.5f}')
    
    print(f"α={alpha_test:.3f} | τ_diff={t_diff:6.1f} | RMSE={rmse_test:.6f}")

plt.tight_layout()
plt.savefig(reports_dir / '02_exp_diffusivity.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n⚠️ Low α (slow diffusion): Sharp gradients persist → harder for PINN")
print("   High α (fast diffusion): Rapid changes in early time → need fine sampling near t=0")

---
## 5. Experiment: Initial Condition Sensitivity

**Question**: How do different ICs affect PINN performance?

In [ ]:
# === Experiment: Different initial conditions ===
ic_types = ['gaussian', 'sine', 'step']
results_ic = []

print("Experiment: Effect of Initial Condition")
print("="*60)

fig, axes = plt.subplots(2, 3, figsize=(14, 8))

for idx, ic_type in enumerate(ic_types):
    set_seed(SEED)
    
    # Define IC function for FD
    if ic_type == 'gaussian':
        u0_fn = lambda x: np.exp(-((x - 0.5)**2) / (2 * 0.1**2))
    elif ic_type == 'sine':
        u0_fn = lambda x: np.sin(np.pi * x)
    elif ic_type == 'step':
        u0_fn = lambda x: np.where(x < 0.5, 1.0, 0.0)
    
    # FD baseline
    u_fd_test = solve_heat_equation_finite_difference(
        alpha=0.01, x=x_fd, t=t_fd,
        u0_fn=u0_fn, bc_left=0.0, bc_right=0.0
    )
    
    # Train PINN
    config_test = HeatEquationConfig(
        alpha=0.01, x_min=0.0, x_max=L, t_max=T_max,
        bc_left=0.0, bc_right=0.0,
        initial_condition=ic_type,
        n_collocation_x=50, n_collocation_t=50,
        hidden_dims=[64, 64, 64],
        epochs=5000,
        lr=1e-3,
    )
    
    pinn_test = HeatEquationPINN(config_test)
    _ = pinn_test.train(verbose=0)
    
    u_pred = pinn_test.predict(x_flat, t_flat).reshape(X_grid.shape)
    rmse_test = np.sqrt(np.mean((u_pred - u_fd_test)**2))
    
    results_ic.append({'ic': ic_type, 'rmse': rmse_test})
    
    # Plot FD solution
    ax = axes[0, idx]
    im = ax.pcolormesh(X_grid, T_grid, u_fd_test, shading='auto', cmap='hot')
    plt.colorbar(im, ax=ax)
    ax.set_title(f'{ic_type.upper()} IC - FD Solution')
    ax.set_xlabel('x'); ax.set_ylabel('t')
    
    # Plot error
    ax = axes[1, idx]
    im = ax.pcolormesh(X_grid, T_grid, np.abs(u_pred - u_fd_test), shading='auto', cmap='Reds')
    plt.colorbar(im, ax=ax)
    ax.set_title(f'PINN Error (RMSE={rmse_test:.4f})')
    ax.set_xlabel('x'); ax.set_ylabel('t')
    
    print(f"{ic_type:10s} | RMSE={rmse_test:.6f}")

plt.tight_layout()
plt.savefig(reports_dir / '02_exp_initial_conditions.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n⚠️ Step function has discontinuity → hardest for PINN (Gibbs-like artifacts)")

---
## 6. Summary Results

In [ ]:
# === Save summary ===
summary = f"""
# PINN PDE: 1D Heat Equation - Results Summary

**Date**: {time.strftime('%Y-%m-%d %H:%M')}
**Seed**: {SEED}

## Problem Setup

- Domain: x ∈ [0, 1], t ∈ [0, 1]
- Thermal diffusivity: α = {alpha}
- Boundary conditions: u(0,t) = u(1,t) = 0
- Initial condition: Gaussian bump

## Baseline Comparison

| Metric | Value |
|--------|-------|
| RMSE | {rmse:.6f} |
| Max Error | {max_error:.6f} |
| Relative Error | {rel_error:.2f}% |
| PINN Time | {pinn_time:.2f}s |
| FD Time | {fd_time:.4f}s |

## Collocation Density Experiment

| N_col | RMSE |
|-------|------|
""" + "\n".join([f"| {r['n_total']} | {r['rmse']:.6f} |" for r in results_sampling]) + """

## Diffusivity Experiment

| α | τ_diff | RMSE |
|---|--------|------|
""" + "\n".join([f"| {r['alpha']:.3f} | {r['t_diff']:.1f} | {r['rmse']:.6f} |" for r in results_alpha]) + """

## Initial Condition Experiment

| IC Type | RMSE |
|---------|------|
""" + "\n".join([f"| {r['ic']} | {r['rmse']:.6f} |" for r in results_ic]) + """

## Key Findings

1. **PINN achieves ~1-5% relative error** depending on problem difficulty
2. **Collocation density**: More points → better accuracy, diminishing returns past 2500
3. **Low diffusivity is hard**: Sharp gradients persist, need more points
4. **Discontinuous ICs are hard**: Step functions cause largest errors
5. **FD is ~100-1000x faster** for this simple problem

## Failure Modes

- Boundary leakage: BC not satisfied (increase λ_BC)
- Long-time drift: Error grows with t (use curriculum or causal weighting)
- Sharp gradients: Gibbs-like oscillations near discontinuities
"""

with open(reports_dir / '02_pinn_pde_summary.md', 'w') as f:
    f.write(summary)

print(summary)
print(f"\n✓ Summary saved to {reports_dir / '02_pinn_pde_summary.md'}")

---
## 7. Failure Modes & Debugging

### Common Issues

| Symptom | Cause | Fix |
|---------|-------|-----|
| BC violated | λ_BC too small | Increase λ_BC or use hard constraints |
| IC not matched | λ_IC too small | Increase λ_IC |
| Error grows with t | Temporal drift | Use causal training or time-windowing |
| Gibbs oscillations | Discontinuous IC | Smooth IC or adaptive sampling near jumps |
| Training stuck | LR too high/low | Use scheduler, try different LRs |

### Debugging Checklist

1. **Plot loss components separately**: Which one is largest?
2. **Check BC/IC satisfaction**: Evaluate u at boundaries and t=0
3. **Visualize residual field**: Where is PDE residual largest?
4. **Inspect collocation points**: Are they well-distributed?
5. **Monitor gradients**: Are they exploding or vanishing?

---
## 8. Mini Exercises

**Exercise 1**: Change to non-zero BCs: u(0,t)=1, u(1,t)=0. What's the steady state?

**Exercise 2**: Double the simulation time (t_max=2.0). Does error grow?

**Exercise 3**: Try a 5-layer network [64,64,64,64,64]. Does it help?

**Exercise 4**: Plot the PDE residual heatmap. Where is error largest?

**Exercise 5**: Implement Latin Hypercube Sampling for collocation points.

In [ ]:
# === Exercise 1: Non-zero BCs ===
# YOUR CODE HERE


In [ ]:
# === Exercise 2: Longer time ===
# YOUR CODE HERE


In [ ]:
# === Exercise 3: Deeper network ===
# YOUR CODE HERE


In [ ]:
# === Exercise 4: Residual visualization ===
# YOUR CODE HERE


In [ ]:
# === Exercise 5: LHS sampling ===
# Hint: from scipy.stats.qmc import LatinHypercube
# YOUR CODE HERE


---
## Solutions

In [ ]:
# === Solution 1: Non-zero BCs ===
# Uncomment to see solution:

# set_seed(42)
# config_ex1 = HeatEquationConfig(
#     alpha=0.01, x_min=0.0, x_max=1.0, t_max=2.0,
#     bc_left=1.0, bc_right=0.0,  # Non-zero left BC
#     initial_condition="gaussian",
#     epochs=5000
# )
# pinn_ex1 = HeatEquationPINN(config_ex1)
# _ = pinn_ex1.train(verbose=0)
#
# # Steady state should be linear: u(x) = 1 - x
# x_test = np.linspace(0, 1, 50)
# t_final = np.full_like(x_test, 2.0)
# u_final = pinn_ex1.predict(x_test, t_final)
#
# plt.figure(figsize=(8, 4))
# plt.plot(x_test, u_final, 'r-', lw=2, label='PINN (t=2.0)')
# plt.plot(x_test, 1 - x_test, 'k--', lw=2, label='Steady state: 1-x')
# plt.xlabel('x'); plt.ylabel('u')
# plt.title('Steady State with Non-zero BCs')
# plt.legend(); plt.show()

---
## Key Takeaways

### ✅ What We Learned

1. **PINNs extend naturally to PDEs** - just add spatial derivatives
2. **Loss balancing is critical**: λ_BC, λ_IC, λ_PDE all matter
3. **Collocation density affects accuracy**: More points help, but diminishing returns
4. **Problem physics affects difficulty**: Low α, discontinuous ICs are hard

### ⚠️ Limitations

1. **Much slower than FD** for simple problems (~100-1000x)
2. **Error can drift in time** without special handling
3. **Sharp gradients cause trouble** (spectral bias)
4. **Hyperparameter sensitive** (λ weights, architecture, LR)

### 💡 When to Use PINNs for PDEs?

- **Inverse problems**: Discover unknown PDE coefficients
- **Irregular geometries**: Where meshing is difficult
- **Multi-physics coupling**: Combining multiple PDEs
- **NOT for**: Simple diffusion where FD/FEM work great!